In [1]:
import duckdb
import pandas as pd

## Objectif : reconstruire une table de prélèvements uniques contenant toutes les caractéristiques possibles des prélèvements à partir de la table d'association prélèvements/UDI

In [2]:
# connection à la db
con = duckdb.connect(database="./../../database/data.duckdb", read_only=True)

In [3]:
# chargement de la table des prélèvements par UDI
prelevements = con.execute("SELECT * FROM edc_prelevements").df()
print(prelevements.shape)
prelevements.head()

(2083345, 20)


,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
0,001,001000003,01007,AMBRONAY,None,None,None,00100119766,2020-02-13,11h40,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-07
1,001,001000003,01007,AMBRONAY,001001304,TTP (CLG) AMBRONAY,100 %,00100120290,2020-03-17,11h15,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-07
2,001,001000003,01007,AMBRONAY,None,None,None,00100120717,2020-05-14,11h54,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-07
3,001,001000003,01007,AMBRONAY,None,None,None,00100121072,2020-06-11,12h00,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-07
4,001,001000003,01007,AMBRONAY,None,None,None,00100121547,2020-07-08,11h09,Eau d'alimentation conforme aux exigences de q...,SI REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,SIE REGION D'AMBERIEU-EN-BUGEY,C,C,C,C,2020,2025-02-07


#### Analyse rapide de l'unicité sur l'association cdreseau/referenceprel

In [4]:
prelevements[prelevements[['cdreseau','referenceprel']].duplicated(keep=False)]

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
1118096,066,066000125,66021,BOMPAS,None,None,None,06600184310,2024-07-02,09h15,Eau d'alimentation non conforme aux exigences ...,CATALANE DES EAUX SECTEUR CENTRE,CATALANE DES EAUX - EAU AGGLO,CATALANE DES EAUX - EAU AGGLO,C,C,C,C,2024,2025-02-07
1532648,066,066000125,66021,BOMPAS,None,None,None,06600184310,2021-07-01,14h32,Eau d'alimentation non conforme aux exigences ...,COM URBAIN PERP MEDITERRANEE VEOLIA,VEOLIA EAU CGE PERPIGNAN,COM URBAINE PERPIGNAN MEDITERRANEE,C,C,C,N,2021,2025-02-07


On a un soucis sur les prélèvements : 2 prélèvements différents possèdent le même ID dans notre base. On va supprimer le plus ancien pour pouvoir continuer l'analyse avec uniquement des associations cdreseau/referenceprel uniques.

In [5]:
index = prelevements[(prelevements['referenceprel'] == '06600184310') & (prelevements['heureprel'] == '14h32')].index[0]
prelevements.drop(index, inplace=True)
print(prelevements.shape)
prelevements[prelevements[['cdreseau','referenceprel']].duplicated(keep=False)]

(2083344, 20)


,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date


Dans la doc officielle, on trouve le paragraphe suivant :  
"A noter que, dans ce jeu de données, l’ensemble des résultats d’analyses sont rapportés à l’unité de distribution concernée, cependant les prélèvements peuvent avoir été effectivement réalisés en amont du réseau de distribution, sur une installation de traitement, de production et de transport de l’eau (TTP), voire sur une installation de ressource en eau ou captage (CAP). En effet, suivant les paramètres concernés ou encore la configuration locale de production d’eau (présence de traitement, caractéristique de celui-ci…), les prélèvements réalisés dans le cadre du contrôle sanitaire le long de la chaîne de production (de la ressource au robinet) peuvent ou non être extrapolés à la qualité de l’eau mise à disposition du consommateur. Cette évaluation est réalisée localement par chaque ARS.

Dans le cas où le prélèvement est réalisé sur une autre installation, certains champs supplémentaires du fichier PLV fournissent des informations sur cette installation : ‘cdreseauamont‘ (code de l’installation amont), ‘nomreseauamont’ (nom de l’installation amont), ‘pourcentdebit’ (pourcentage de débit de l’installation amont : indique si l’installation amont alimente exclusivement ou en partie cette UDI). Lorsque les prélèvements ont été effectivement réalisés sur l’UDI concernée, ces champs sont vides."  

Traduction : en théorie, on devrait avoir une ligne par prélèvement et par UDI. Lorsque le prélèvement a été effectué en amont d'une UDI, et que l'ARS a extrapolé la qualité de l'eau à une UDI en aval du point de prélèvement, la ligne de prélèvement contient les informations sur le réseau en amont sur lequel le prélèvement a été effectué. Ce qui veut dire que pour un même prélèvement, on peut avoir plusieurs UDI rattachées à ce prélèvement, si l'ARS a fait l'extrapolation.

On va analyser ces cas d'extrapolation.

In [6]:
# on récupère tous les prélèvements qui sont non uniques
prelevements[prelevements['referenceprel'].duplicated(keep=False)].sort_values('referenceprel').head()

,cddept,cdreseau,inseecommuneprinc,nomcommuneprinc,cdreseauamont,nomreseauamont,pourcentdebit,referenceprel,dateprel,heureprel,conclusionprel,ugelib,distrlib,moalib,plvconformitebacterio,plvconformitechimique,plvconformitereferencebact,plvconformitereferencechim,de_partition,de_ingestion_date
1276,001,001000488,01032,BELIGNEUX,001000487,BELIGNEUX LA VALBONNE,0 %,00100119381,2020-01-23,10h45,Eau d'alimentation conforme aux exigences de q...,CC COTIERE A MONTLUEL (3CM) SUEZ,SUEZ EAU FRANCE,CC DE LA COTIERE A MONTLUEL (3CM),C,C,C,C,2020,2025-02-07
1261,001,001000487,01032,BELIGNEUX,None,None,None,00100119381,2020-01-23,10h45,Eau d'alimentation conforme aux exigences de q...,CC COTIERE A MONTLUEL (3CM) SUEZ,SUEZ EAU FRANCE,CC DE LA COTIERE A MONTLUEL (3CM),C,C,C,C,2020,2025-02-07
1262,001,001000487,01032,BELIGNEUX,001000488,BELIGNEUX CHANES,100 %,00100119382,2020-01-23,12h08,Eau d'alimentation conforme aux exigences de q...,CC COTIERE A MONTLUEL (3CM) SUEZ,SUEZ EAU FRANCE,CC DE LA COTIERE A MONTLUEL (3CM),C,C,C,C,2020,2025-02-07
1277,001,001000488,01032,BELIGNEUX,None,None,None,00100119382,2020-01-23,12h08,Eau d'alimentation conforme aux exigences de q...,CC COTIERE A MONTLUEL (3CM) SUEZ,SUEZ EAU FRANCE,CC DE LA COTIERE A MONTLUEL (3CM),C,C,C,C,2020,2025-02-07
3304,001,001001032,01033,VALSERHONE,001001028,CHATILLON HAUT SERVICE,0 %,00100119386,2020-01-15,10h27,Eau d'alimentation conforme aux exigences de q...,CC PAYS BELLEGARDIEN (CCPB),CC PAYS BELLEGARDIEN-REGIE DE L'EAU,CC PAYS BELLEGARDIEN-REGIE DE L'EAU,C,C,C,C,2020,2025-02-07


Si on analyse les 2 premières lignes, on retrouve en ligne 1 un prélèvement qui correspond à la description faite dans la doc : un prélèvement extrapolé à une UDI (réseau 001000488) dont le prélèvement à été fait en amont (réseau 001000487) ET en ligne 2, le même prélèvement associé au réseau en amont (réseau 001000487), ce qui veut dire que le réseau en amont est également une UDI.

Ce dataset représente donc la liste des associations entre les prélèvements (effectués sur une UDI ou non) et le réseau du point de prélèvement si c'est une UDI, ainsi que tous les réseaux UDI en aval du point de prélèvement que l'ARS locale a décidé d'extrapoler comme partageant la même qualité d'eau que le réseau d'origine.

On peut donc déduire que si une association UDI/prélèvement n'a pas de réseau amont renseigné, le prélèvement a eu lieu dans l'UDI, sinon le prélèvement a eu lieu dans un réseau en amont.

On va donc établir la liste des prélèvements et le réseau dans lequel le prélèvement a eu lieu afin de faire de l'analyse sur les prélèvements uniquement.

In [7]:
# Cas 1 : le prélèvement a eu lieu dans une UDI
# on conserve que les colonnes concernant l'UDI et le prélèvement et en mettant de coté les conclusions qui ne nous intéressent pas pour le moment
liste_col = ['cddept','inseecommuneprinc','nomcommuneprinc','referenceprel','dateprel','heureprel','cdreseau']
prelevements_dans_UDI = prelevements.loc[prelevements['cdreseauamont'].isna(),liste_col].copy()

# Cas 2 : le prélèvement a eu lieu dans un réseau amont qui n'est pas une UDI, ie dans un réseau amont qui n'est pas dans la liste des UDI ayant un prélèvement
# on conserve que les colonnes concernant le réseau amont et le prélèvement et en mettant de coté les conclusions également
liste_col_2 = ['cddept','inseecommuneprinc','nomcommuneprinc','referenceprel','dateprel','heureprel','cdreseauamont','nomreseauamont']
df_UDI_ayant_prelevement = prelevements_dans_UDI[['cdreseau','referenceprel']]
r = prelevements[prelevements['cdreseauamont'].isna() == False]\
.merge(df_UDI_ayant_prelevement, left_on=['cdreseauamont','referenceprel'], right_on=['cdreseau','referenceprel'], how='left',indicator=True)
prelevements_hors_UDI = r.loc[r['_merge'] == 'left_only',liste_col_2].drop_duplicates()

# note : on ne peut pas conserver le champ 'pourcentdebit' car c'est une caractéristique de l'association 'reseau/reseau amont' et on ne conserve ici que
# les informations du réseau dans lequel a eu lieu le prélèvement

Si toutes les assomptions sont correctes, on ne devrait avoir que des valeurs uniques de prélèvement dans les 2 cas, et la somme des lignes de chaque subset devrait être égale au nombre de valeurs uniques des références de prélèvements. On vérifie.

In [8]:
print(f'Nombre de prélèvements dans UDI non uniques : {prelevements_dans_UDI['referenceprel'].duplicated().sum()}')
print(f'Nombre de prélèvements hors UDI non uniques : {prelevements_hors_UDI['referenceprel'].duplicated().sum()}')
val = prelevements_dans_UDI.shape[0] + prelevements_hors_UDI.shape[0] == len(prelevements['referenceprel'].unique())
print(f'Nombre total de prélèvements égal aux nombres de prélèvements uniques : {val}')

Nombre de prélèvements dans UDI non uniques : 0
Nombre de prélèvements hors UDI non uniques : 0
Nombre total de prélèvements égal aux nombres de prélèvements uniques : True


Tout est ok. On peut concaténer les tables pour obtenir une table des prélèvements uniques avec le type de réseau.

In [9]:
prelevements_uniques = pd.concat([prelevements_dans_UDI,prelevements_hors_UDI], axis=0).reset_index(drop=True)
prelevements_uniques

,cddept,inseecommuneprinc,nomcommuneprinc,referenceprel,dateprel,heureprel,cdreseau,cdreseauamont,nomreseauamont
0,001,01007,AMBRONAY,00100119766,2020-02-13,11h40,001000003,NaN,NaN
1,001,01007,AMBRONAY,00100120717,2020-05-14,11h54,001000003,NaN,NaN
2,001,01007,AMBRONAY,00100121072,2020-06-11,12h00,001000003,NaN,NaN
3,001,01007,AMBRONAY,00100121547,2020-07-08,11h09,001000003,NaN,NaN
4,001,01007,AMBRONAY,00100122035,2020-08-13,10h44,001000003,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1436109,976,97606,CHIRONGUI,97600028815,2022-06-28,11h56,NaN,976003502,STATION CHIRONGUI
1436110,976,97606,CHIRONGUI,97600028991,2022-08-30,11h37,NaN,976003502,STATION CHIRONGUI
1436111,976,97606,CHIRONGUI,97600029112,2022-10-04,08h56,NaN,976003502,STATION CHIRONGUI
1436112,976,97606,CHIRONGUI,97600029263,2022-12-07,09h54,NaN,976003502,STATION CHIRONGUI


On a maintenant un dataframe qui contient une liste de prélèvements uniques

Dans ce nouveau dataframe, on peut facilement identifier les prélèvements qui ont eu lieu dans une UDI et ceux en amont des UDI : si le champ cdreseau est renseigné, c'est une UDI, sinon c'est un champ en amont qui n'est pas une UDI.

## CONCLUSION

Pour pouvoir établir la liste des prélèvements uniques à partir de la table edc_prelevements, il faut :  
1°) établir la liste des prélèvements qui ont eu lieux dans une UDI, cad par exemple les lignes qui n'ont pas de valeur 'cdreseauamont' renseigné  
2°) établir la liste des prélèvements qui ont eu lieux dans un réseau amont qui n'est pas une UDI, cad les lignes qui ont une paire de valeurs 'cdreseauamont'/'referenceprel' qui n'apparait pas dans la liste 1°)  
3°) concaténer les 2 listes  

Un exemple pour obtenir un résultat équivalent en SQL (en ignorant le prélèvement en doublon) :

In [10]:
table_prel_uniques = con.execute("""
SELECT cddept, inseecommuneprinc, nomcommuneprinc, referenceprel, dateprel, heureprel, cdreseau, 'UDI' as type_reseau
FROM edc_prelevements
WHERE cdreseauamont IS NULL AND (referenceprel != '06600184310' OR heureprel != '14h32')

UNION

SELECT DISTINCT edc_prelevements.cddept,
                edc_prelevements.inseecommuneprinc,
                edc_prelevements.nomcommuneprinc,
                edc_prelevements.referenceprel,
                edc_prelevements.dateprel,
                edc_prelevements.heureprel,
                edc_prelevements.cdreseauamont,
                'non-UDI' as type_reseau
FROM edc_prelevements
LEFT JOIN (SELECT cddept, inseecommuneprinc, nomcommuneprinc, referenceprel, dateprel, heureprel, cdreseau
          FROM edc_prelevements 
          WHERE cdreseauamont IS NULL AND (referenceprel != '06600184310' OR heureprel != '14h32')
          ) as prel2 on edc_prelevements.cdreseauamont = prel2.cdreseau AND edc_prelevements.referenceprel = prel2.referenceprel
WHERE edc_prelevements.cdreseauamont IS NOT NULL AND prel2.cdreseau IS NULL
""").df()
table_prel_uniques

,cddept,inseecommuneprinc,nomcommuneprinc,referenceprel,dateprel,heureprel,cdreseau,type_reseau
0,087,87009,BEAUMONT-DU-LAC,08700105467,2020-07-08,10h50,087000860,UDI
1,087,87198,VAULRY,08700104852,2020-06-10,08h45,087000930,UDI
2,087,87198,VAULRY,08700105059,2020-06-22,11h55,087000930,UDI
3,087,87198,VAULRY,08700106258,2020-09-14,09h00,087000930,UDI
4,087,87050,COUZEIX,08700105005,2020-06-18,14h10,087000949,UDI
...,...,...,...,...,...,...,...,...
1436109,057,57091,BLIESBRUCK,05700155465,2022-03-22,10h23,057002078,non-UDI
1436110,057,57225,FONTENY,05700159438,2022-10-28,08h26,057002100,non-UDI
1436111,057,57651,SIERSTHAL,05700155225,2022-03-11,10h06,057002105,non-UDI
1436112,057,57445,MARIEULLES,05700156682,2022-05-24,10h37,057004217,non-UDI


On vérifie qu'on a bien une table contenant tous les prélèvements uniques.

In [11]:
liste_prel_unique = con.execute(""" SELECT DISTINCT referenceprel FROM edc_prelevements""").df()
print(liste_prel_unique.values.sort() == table_prel_uniques['referenceprel'].values.sort())

True


In [12]:
con.close()